In [1]:
import os
import time
import datetime

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt


import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
from nltk.translate.bleu_score import sentence_bleu
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

In [20]:
import string, re
test_set = pd.read_csv('medialog_eng.csv')
print(f"testset shape - {test_set.shape}")
regex = re.compile('[%s]' % re.escape(string.punctuation))


def process_string(answer):
    p_answer = answer.translate(str.maketrans('','', string.punctuation))
    curr_ans = p_answer.split(' ')
    return curr_ans
    

testset shape - (604, 2)


In [26]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium

path = 'C:/Users/wjtay/Documents/GitHub/medicalbot/gpt_models/model_save_3'
model =  GPT2LMHeadModel.from_pretrained('C:/Users/wjtay/Documents/GitHub/medicalbot/gpt_models/model_save_3')
model.resize_token_embeddings(len(tokenizer))

model.eval()




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:


def get_prediction(n, query):
    prompt = f"<|startoftext|><|question|>{query}<|answer|>"

    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    # generated = generated.to(device)

    # print(generated)

    sample_outputs = model.generate(
                                    generated, 
                                    #bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 300,
                                    top_p=0.95, 
                                    num_return_sequences=3
                                    )
    decoded_outputs = []

    for i, sample_output in enumerate(sample_outputs):
        # print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
        decoded_outputs.append(tokenizer.decode(sample_output, skip_special_tokens=True))
    
    
    # print(decoded_outputs)
    return decoded_outputs


In [30]:
from nltk.translate.bleu_score import sentence_bleu
all_bleu_score = []

for idx, row in test_set.iterrows():
    question = row["question"]
    answer = process_string(row["answer"])
    predicteds = get_prediction(3, question)

    curr_bleu_score = 0
    for output in predicteds:
        predict = output.split('<|answer|>')[1]
        curr_bleu_score = max(curr_bleu_score, sentence_bleu(answer, process_string(predict)))
        print(f'bleu: {curr_bleu_score}')
        # print(f'ans: {answer}')
        # print(f'predict: {process_string(predict)}')
    
    all_bleu_score.append(curr_bleu_score)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Python38\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Python38\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Python38\lib\site-packages\nltk\translate\bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or

bleu: 7.752349855101862e-232
bleu: 7.850707993042515e-232
bleu: 7.850707993042515e-232


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bleu: 7.544641259221007e-232
bleu: 7.544641259221007e-232
bleu: 7.544641259221007e-232


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bleu: 8.087494814426946e-232
bleu: 8.087494814426946e-232
bleu: 8.301113029505914e-232


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


bleu: 7.817228731469945e-232
bleu: 7.817228731469945e-232
bleu: 7.817228731469945e-232


KeyboardInterrupt: 